In [1]:

%pprint

Pretty printing has been turned OFF



---
# Load needed libraries and functions

In [2]:

from datetime import datetime
import humanize
import os
import sys
import time
import warnings
import winsound

warnings.filterwarnings('ignore')
duration = 1000  # milliseconds
freq = 880  # Hz

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [6]:

t0 = time.time()

# Get the Neo4j driver
from storage import Storage
s = Storage(
    data_folder_path=os.path.abspath('../data'),
    saves_folder_path=os.path.abspath('../saves')
)

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(s=s, verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(
    s=s,
    secrets_json_path=os.path.abspath('../data/secrets/jh_secrets.json')
)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

# Get the neo4j object
from cypher_utils import CypherUtilities
cu = CypherUtilities(
    uri=uri, user=user, password=password, driver=None, s=s, ha=ha
)

from section_classifier_utils import SectionLRClassifierUtilities, SectionCRFClassifierUtilities
slrcu = SectionLRClassifierUtilities(ha=ha, cu=cu, verbose=False)
scrfcu = SectionCRFClassifierUtilities(cu=cu, ha=ha, verbose=False)

from section_utils import SectionUtilities
su = SectionUtilities(wsu=wsu, ihu=None, hc=None, crf=None, slrcu=slrcu, verbose=False)

duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
print(f'Utility libraries created in {duration_str}')
print(f'Last run on {datetime.now()}')

Utility libraries created in 0 seconds
Last run on 2023-03-22 12:49:48.283907



----

In [34]:

try:
    driver = wsu.get_driver()
    winsound.Beep(freq, int(duration/2))
    wsu.log_into_indeed(driver, verbose=False)
finally:
    file_path = '../data/html/indeed_email.html'
    page_soup = wsu.get_page_soup(file_path)
    css_selector = 'table > tbody > tr > td > a > table > tbody > tr > td > a'
    link_soups_list = page_soup.select(css_selector)
    
    # Get rid of the duplicate URLs
    url_strs_set = set()
    for link_soup in link_soups_list:
        url_str = link_soup['href']
        url_strs_set.add(url_str)
    
    display(len(url_strs_set))

Getting the FireFox driver
Waiting for the web_element to show up: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


22


## Check the back Firefox window to make sure you logged in correctly and don't have any messages to read

In [35]:

files_list = []
for url_str in url_strs_set:
    file_node_dict, files_list = su.load_indeed_posting_url(viewjob_url=url_str, driver=driver,
                                                            files_list=files_list, verbose=False)
print(f'Fileing {len(files_list)} postings complete. Delete the email.')

Fileing 7 postings complete. Delete the email.


In [36]:

import os

try:
    driver.close()
except Exception as e:
    print(f'{e.__class__} error: {str(e).strip()}')
cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
winsound.Beep(freq, duration)
files_list

['97e424f0aa71f24c_Senior_Data_Analyst_New_York_NY_Indeed_com.html', '8087fc322bae24ed_Python_SQL_Remote_Indeed_com.html', '53931aca20b3fd9d_Senior_Python_Developer_Remote_Indeed_com.html', 'd3f5a60a709a9786_Software_Engineer_Data_San_Francisco_CA_Indeed_com.html', '3dc25bd0e4058b4b_Machine_Learning_Engineer_Remote_Indeed_com.html', 'ff212192f3dab37e_Data_Engineer_w2_Remote_Indeed_com.html', '5494ded05fb91269_Data_Science_Practice_Lead_Health_IT_Remote_Indeed_com.html']